# Setting up environment

In [1]:
# Khoi Hoang
# hoanganhkhoil@gmail.com
# Neural Network Editor

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from copy import copy

import inspect
import time

import numpy as np
import tensorflow as tf

import reader
import os


data_path = "simple-examples/data/"
save_path = "save/"

def data_type():
  return tf.float32


class PTBInput(object):
  """The input data."""

  def __init__(self, config, data, name=None):
    self.batch_size = batch_size = config.batch_size
    self.num_steps = num_steps = config.num_steps
    self.epoch_size = ((len(data) // batch_size) - 1) // num_steps
    self.input_data, self.targets = reader.ptb_producer(
        data, batch_size, num_steps, name=name)


class PTBModel(object):
  """The PTB model."""

  def __init__(self, is_training, config, input_):
    self._input = input_

    batch_size = input_.batch_size
    num_steps = input_.num_steps
    size = config.hidden_size
    vocab_size = config.vocab_size

    def lstm_cell():
      if 'reuse' in inspect.getargspec(
          tf.contrib.rnn.BasicLSTMCell.__init__).args:
        return tf.contrib.rnn.BasicLSTMCell(
            size, forget_bias=0.0, state_is_tuple=True,
            reuse=tf.get_variable_scope().reuse)
      else:
        return tf.contrib.rnn.BasicLSTMCell(
            size, forget_bias=0.0, state_is_tuple=True)
    attn_cell = lstm_cell
    
    if is_training and config.keep_prob < 1:
      def attn_cell():
        return tf.contrib.rnn.DropoutWrapper(
            lstm_cell(), output_keep_prob=config.keep_prob)
    cell = tf.contrib.rnn.MultiRNNCell(
        [attn_cell() for _ in range(config.num_layers)], state_is_tuple=True)

    self._initial_state = cell.zero_state(batch_size, data_type())

    with tf.device("/cpu:0"):
      embedding = tf.get_variable(
          "embedding", [vocab_size, size], dtype=data_type())
      inputs = tf.nn.embedding_lookup(embedding, input_.input_data)

    if is_training and config.keep_prob < 1:
      inputs = tf.nn.dropout(inputs, config.keep_prob)

    outputs = []
    state = self._initial_state
    
    with tf.variable_scope("RNN"):
      for time_step in range(num_steps):
        if time_step > 0: tf.get_variable_scope().reuse_variables()
        (cell_output, state) = cell(inputs[:, time_step, :], state)
        outputs.append(cell_output)

    output = tf.reshape(tf.stack(axis=1, values=outputs), [-1, size])
    softmax_w = tf.get_variable(
        "softmax_w", [size, vocab_size], dtype=data_type())
    softmax_b = tf.get_variable("softmax_b", [vocab_size], dtype=data_type())
    logits = tf.matmul(output, softmax_w) + softmax_b
    self._output_probs = tf.nn.softmax(logits)


    # Reshape logits to be 3-D tensor for sequence loss
    logits = tf.reshape(logits, [batch_size, num_steps, vocab_size])

    # use the contrib sequence loss and average over the batches
    loss = tf.contrib.seq2seq.sequence_loss(
        logits,
        input_.targets,
        tf.ones([batch_size, num_steps], dtype=data_type()),
        average_across_timesteps=False,
        average_across_batch=True
    )

    # update the cost variables
    self._cost = cost = tf.reduce_sum(loss)
    self._final_state = state

    if not is_training:
      return

    self._lr = tf.Variable(0.0, trainable=False)
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars),
                                      config.max_grad_norm)
    optimizer = tf.train.GradientDescentOptimizer(self._lr)
    self._train_op = optimizer.apply_gradients(
        zip(grads, tvars),
        global_step=tf.contrib.framework.get_or_create_global_step())

    self._new_lr = tf.placeholder(
        tf.float32, shape=[], name="new_learning_rate")
    self._lr_update = tf.assign(self._lr, self._new_lr)

  def assign_lr(self, session, lr_value):
    session.run(self._lr_update, feed_dict={self._new_lr: lr_value})

  @property
  def input(self):
    return self._input

  @input.setter
  def input(self, value):
    self._input = value

  @property
  def initial_state(self):
    return self._initial_state

  @property
  def cost(self):
    return self._cost

  @property
  def final_state(self):
    return self._final_state

  @property
  def lr(self):
    return self._lr

  @property
  def train_op(self):
    return self._train_op

  @property
  def output_probs(self):
    return self._output_probs


class SmallConfig(object):
  """Small config."""
  init_scale = 0.1
  learning_rate = 1.0
  max_grad_norm = 5
  num_layers = 2
  num_steps = 20
  hidden_size = 200
  max_epoch = 4
  max_max_epoch = 13 # 13
  keep_prob = 0.8   # 1.0
  lr_decay = 0.5
  batch_size = 20
  vocab_size = 21028     # 28614


class MediumConfig(object):
  """Medium config."""
  init_scale = 0.05
  learning_rate = 1.0
  max_grad_norm = 5
  num_layers = 2
  num_steps = 35
  hidden_size = 650
  max_epoch = 6
  max_max_epoch = 39
  keep_prob = 0.5
  lr_decay = 0.8
  batch_size = 20
  vocab_size = 21028


class LargeConfig(object):
  """Large config."""
  init_scale = 0.04
  learning_rate = 1.0
  max_grad_norm = 10
  num_layers = 2
  num_steps = 35
  hidden_size = 1500
  max_epoch = 14
  max_max_epoch = 55
  keep_prob = 0.35
  lr_decay = 1 / 1.15
  batch_size = 20
  vocab_size = 21028


class TestConfig(object):
  """Tiny config, for testing."""
  init_scale = 0.1
  learning_rate = 1.0
  max_grad_norm = 1
  num_layers = 1
  num_steps = 2
  hidden_size = 2
  max_epoch = 1
  max_max_epoch = 1
  keep_prob = 1.0
  lr_decay = 0.5
  batch_size = 20
  vocab_size = 21028


def run_epoch(session, model, eval_op=None, verbose=False):
  """Runs the model on the given data."""
  start_time = time.time()
  costs = 0.0
  iters = 0
  state = session.run(model.initial_state)

  fetches = {
      "cost": model.cost,
      "final_state": model.final_state,
  }
  if eval_op is not None:
    fetches["eval_op"] = eval_op

  for step in range(model.input.epoch_size):
    feed_dict = {}
    for i, (c, h) in enumerate(model.initial_state):
      feed_dict[c] = state[i].c
      feed_dict[h] = state[i].h

    vals = session.run(fetches, feed_dict)
    cost = vals["cost"]
    state = vals["final_state"]

    costs += cost
    iters += model.input.num_steps

    if verbose and step % (model.input.epoch_size // 10) == 10:
      print("%.3f perplexity: %.3f speed: %.0f wps" %
            (step * 1.0 / model.input.epoch_size, np.exp(costs / iters),
             iters * model.input.batch_size / (time.time() - start_time)))

  return np.exp(costs / iters)

id_to_word = reader.get_id_to_word(data_path)
word_to_id = reader.get_word_to_id(data_path)

def gen_text_2(session, mout, last_state, id_to_word, word_to_id, word):
#     id_to_word = reader.get_id_to_word(data_path)
#     word_to_id = reader.get_word_to_id(data_path)

    if word in word_to_id.keys():
        inputt = np.matrix([[word_to_id[word]]])
    else:
        inputt = np.matrix([[0]])

    output_probs, state = session.run([mout.output_probs, mout.final_state],
                                    {mout.input.input_data: inputt, mout.initial_state: last_state})
    x = np.argmax(output_probs)

    text = id_to_word[x]

      
    return text, state
  

if not data_path:
    raise ValueError("Must set --data_path to PTB data directory")

raw_data = reader.ptb_raw_data(data_path)
train_data, valid_data, test_data, _ = raw_data
print ("Len vocabulary: ", len(_))

config = SmallConfig()

eval_config = SmallConfig()
eval_config.batch_size = 1
eval_config.num_steps = 1

gen_config = SmallConfig()
gen_config.num_steps = 1
gen_config.batch_size = 1

# with tf.Graph().as_default():
initializer = tf.random_uniform_initializer(-config.init_scale,
                                            config.init_scale)

with tf.name_scope("Train"):
  train_input = PTBInput(config=config, data=train_data, name="TrainInput")
  with tf.variable_scope("Model", reuse=None, initializer=initializer):
    m = PTBModel(is_training=True, config=config, input_=train_input)
  tf.summary.scalar("Training Loss", m.cost)
  tf.summary.scalar("Learning Rate", m.lr)

with tf.name_scope("Valid"):
  valid_input = PTBInput(config=config, data=valid_data, name="ValidInput")
  with tf.variable_scope("Model", reuse=True, initializer=initializer):
    mvalid = PTBModel(is_training=False, config=config, input_=valid_input)
  tf.summary.scalar("Validation Loss", mvalid.cost)

with tf.name_scope("Test"):
  test_input = PTBInput(config=eval_config, data=test_data, name="TestInput")
  with tf.variable_scope("Model", reuse=True, initializer=initializer):
    mtest = PTBModel(is_training=False, config=eval_config,
                     input_=test_input)
with tf.name_scope("Generate"):
    rnn_input = PTBInput(config=gen_config, data=train_data, name="GenOut")
    with tf.variable_scope("Model", reuse=True, initializer=initializer):
        mout = PTBModel(is_training=False, config=gen_config, input_=rnn_input)

saver = tf.train.Saver()

#     sv = tf.train.Supervisor(logdir=save_path)
#     saver = tf.train.Saver()
#     with sv.managed_session() as session:
#     with tf.Session() as session:
#         saver.restore(session, tf.train.latest_checkpoint(save_path))
#         print ("Model restored")

#         text_output = gen_text_2(session, mout, id_to_word, word_to_id, "agreement")
#         print (text_output)
#         print ("done")






Len word_to_id:  21028
{'the': 0, 'of': 1, 'to': 2, 'and': 3, 'or': 4, 'in': 5, 'shall': 6, 'a': 7, 'any': 8, 'for': 9, 'be': 10, 'by': 11, 'this': 12, 'agreement': 13, 'such': 14, 'with': 15, 'as': 16, 's': 17, 'that': 18, 'will': 19, 'party': 20, 'is': 21, 'other': 22, 'all': 23, 'product': 24, '1': 25, 'not': 26, '2': 27, 'its': 28, 'on': 29, 'at': 30, '*': 31, 'which': 32, 'from': 33, '3': 34, 'products': 35, 'may': 36, 'parties': 37, 'under': 38, 'if': 39, 'confidential': 40, 'section': 41, '4': 42, '***': 43, 'are': 44, 'no': 45, 'each': 46, 'information': 47, 'has': 48, 'an': 49, 'have': 50, 'date': 51, 'b': 52, '5': 53, 'manufacturing': 54, 'time': 55, 'provided': 56, '<page>': 57, 'upon': 58, 'purchase': 59, 'terms': 60, 'been': 61, 'use': 62, 'including': 63, 'order': 64, 'customer': 65, 'written': 66, '6': 67, 'it': 68, 'set': 69, 'notice': 70, 'days': 71, 'supplier': 72, 'services': 73, 'company': 74, 'treatment': 75, 'contract': 76, 'within': 77, 'delivery': 78, 'forth': 7

Len vocabulary:  21028


/Users/khoihoang/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:51: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()


INFO:tensorflow:Summary name Training Loss is illegal; using Training_Loss instead.
INFO:tensorflow:Summary name Learning Rate is illegal; using Learning_Rate instead.
INFO:tensorflow:Summary name Validation Loss is illegal; using Validation_Loss instead.


# Restore saved model from Tensorflow

In [2]:
session = tf.Session()
saver.restore(session, tf.train.latest_checkpoint(save_path))
print ("Model restored")




INFO:tensorflow:Restoring parameters from save/-43992
Model restored


# Running the application

In [3]:
from tkinter import*
import random
from tkinter.scrolledtext import ScrolledText
textpridict="123"
k=20
num=500

counter = 0
text_=""
state = session.run(mout.initial_state)

def get_text(event):
    global counter
    global state
    global text_
    if counter==0:
        state = session.run(mout.initial_state)
        counter += 1 
    
    text_output, new_state = gen_text_2(session, mout, state, id_to_word, word_to_id, get_last_word(text.get("1.0",'end-1c')))
    
    state = new_state
    text_= text_output + " "
    res.configure(text =text_output)
    
    return state
    
def pridict( myVar):
    #str(entry.get())
    print (myVar)
    num_copy = num
    num_copy += 5
def add_text(event):
    global text_
    text.insert(END,text_)
    get_text(event)
    
def get_last_word(input):
    input_split=str(input).split()
    return input_split[-1] 
w = Tk()
Label(w, text="Neural Network Editor",font=(None, 35),width=70,background='snow2').pack(side="top")

text=Text(w,width=200,height =15)
#just use diffrent bind commands to call or pass diffrent functions 
text.configure(background='snow')
text.bind("<space>",get_text)
text.bind("<Up>",add_text)
text.pack()
w.configure(background='snow2')
w.minsize(width=800, height=700)
res = Label(w,font=(None, 25))
res.configure(background='snow2')

res.pack(side="top")
w.mainloop()


# Visualization 

# <img src="photos/1.png">

# <img src="photos/2.png">

# <img src="photos/3.png">